# **Women cloth Reviews Prediction with Multi Nomial Naive Bayes**

**The multinomial Naive Bayes classifier is suitable for classification with discrete features(e.g., word counts for text xlassification). The multinomial distribution normally requires integer feature counts. However , in practice , fractional counts such as tf-idf may also work.**

# Import library

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn  as sns

# Import dataset

In [68]:
df = pd.read_csv('https://raw.githubusercontent.com/YBI-Foundation/Teaching-Data/main/Women%20Clothing%20E-Commerce%20Review.csv')

In [69]:
df.head()

,Clothing ID,Age,Title,Review,Rating,Recommended,Positive Feedback,Division,Department,Category
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Clothing ID        23486 non-null  int64 
 1   Age                23486 non-null  int64 
 2   Title              19676 non-null  object
 3   Review             22641 non-null  object
 4   Rating             23486 non-null  int64 
 5   Recommended        23486 non-null  int64 
 6   Positive Feedback  23486 non-null  int64 
 7   Division           23472 non-null  object
 8   Department         23472 non-null  object
 9   Category           23472 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.8+ MB


In [71]:
df.describe()

,Clothing ID,Age,Rating,Recommended,Positive Feedback
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,918.118709,43.198544,4.196032,0.822362,2.535936
std,203.298980,12.279544,1.110031,0.382216,5.702202
min,0.000000,18.000000,1.000000,0.000000,0.000000
25%,861.000000,34.000000,4.000000,1.000000,0.000000
50%,936.000000,41.000000,5.000000,1.000000,1.000000
75%,1078.000000,52.000000,5.000000,1.000000,3.000000
max,1205.000000,99.000000,5.000000,1.000000,122.000000


In [72]:
df.shape

(23486, 10)

# **Missing values**

Remove Missing values in Reviews columns with No Review text

In [73]:
df.isna().sum()

Clothing ID             0
Age                     0
Title                3810
Review                845
Rating                  0
Recommended             0
Positive Feedback       0
Division               14
Department             14
Category               14
dtype: int64

In [74]:
df['Review'].fillna("No Review" , inplace = True)

In [75]:
df.isna().sum()

Clothing ID             0
Age                     0
Title                3810
Review                  0
Rating                  0
Recommended             0
Positive Feedback       0
Division               14
Department             14
Category               14
dtype: int64

In [76]:
df['Review']

0        Absolutely wonderful - silky and sexy and comf...
1        Love this dress!  it's sooo pretty.  i happene...
2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review, Length: 23486, dtype: object

# **Define target(y) and features (X)**

In [77]:
df.columns

Index(['Clothing ID', 'Age', 'Title', 'Review', 'Rating', 'Recommended',
       'Positive Feedback', 'Division', 'Department', 'Category'],
      dtype='object')

In [78]:
X= df['Review']

In [79]:
y = df['Rating']

In [80]:
df['Rating'].value_counts()

5    13131
4     5077
3     2871
2     1565
1      842
Name: Rating, dtype: int64

# **Recategories Ratings as Poor(0) and Good(1)**

**Re-Rating 1 ,2,3 as 0 and 4,5 as 1**

In [81]:
df.replace({'Rating' : {1: 0 , 2: 0 , 3: 0 , 4 : 1 , 5: 1}}, inplace = True)

In [82]:
y = df['Rating']

In [83]:
X = df['Review']

# **Train Test Split**

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
X_train , X_test , y_train , y_test = train_test_split(X, y , train_size = 0.7 , stratify = y , random_state = 2529)

In [86]:
X_train.shape , X_test.shape , y_train.shape , y_test.shape

((16440,), (7046,), (16440,), (7046,))

# **Get feature text conversion to tokens**

In [87]:
from sklearn.feature_extraction.text import CountVectorizer

In [88]:
cv = CountVectorizer(lowercase = True , analyzer = 'word' , ngram_range= (2, 3), stop_words = 'english' , max_features = 5000)

In [89]:
X_train = cv.fit_transform(X_train)

In [90]:
cv.get_feature_names_out()

array(['10 12', '10 dress', '10 fit', ..., 'yellow color', 'yes runs',
       'yoga pants'], dtype=object)

In [91]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [92]:
X_test = cv.fit_transform(X_test)

In [93]:
cv.get_feature_names_out()

array(['00 petite', '0p fit', '10 12', ..., 'years come', 'yoga pants',
       'zipper little'], dtype=object)

In [94]:
X_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# **Get Model Train**

In [95]:
from sklearn.naive_bayes import MultinomialNB

In [96]:
model = MultinomialNB()

In [97]:
model.fit(X_train , y_train)

MultinomialNB()

# **Get Model Prediction**

In [98]:
y_pred = model.predict(X_test)

In [99]:
y_pred.shape

(7046,)

In [100]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

# **Get Probability of Each Predicted Class**

In [101]:
model.predict_proba(X_test)

array([[5.43140171e-02, 9.45685983e-01],
       [1.58169512e-01, 8.41830488e-01],
       [7.31367350e-03, 9.92686326e-01],
       ...,
       [3.92486843e-04, 9.99607513e-01],
       [2.24756691e-01, 7.75243309e-01],
       [5.78360255e-02, 9.42163975e-01]])

# **Get Model Evaluation**

In [102]:
from sklearn.metrics import confusion_matrix , classification_report

In [103]:
confusion_matrix(y_test , y_pred)

array([[ 449, 1134],
       [ 989, 4474]])

In [104]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.31      0.28      0.30      1583
           1       0.80      0.82      0.81      5463

    accuracy                           0.70      7046
   macro avg       0.56      0.55      0.55      7046
weighted avg       0.69      0.70      0.69      7046

